In [1]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#adaptacion
from utils.adaptacion import create_SDA

# clasificadores
from sklearn.svm import SVC
from utils.clasificacion import *

import os
import time
import pandas as pd
import numpy as np

#variables para guardar los resultados
tipo = pruebas[1]
dataset_name = datasets[1]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Dataset de Twitter

In [2]:
print tipo
print dataset_name
print data_path

msda
twitter
data


In [3]:
# cargando dataset de Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

In [4]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

Todos los datos disponibles obtenidos


In [5]:
#autoencoder, encoder = create_SDA(784, [128, 64, 32, 16, 8])
print "Creando Stacked Autoencoder..."
autoencoder, encoder = create_SDA(dims, [1000])
print "Stacked Autoencoder creado."

Creando Stacked Autoencoder...
Stacked Autoencoder creado.


In [6]:
X_train = X[:, :dims]

#TODO: resolver lo de validation data
autoencoder.fit(X_train, X_train,
               epochs=50,
               batch_size=256,
               shuffle=True,
               validation_data=(X_train, X_train))

Train on 1124 samples, validate on 1124 samples
Epoch 1/50
1124/1124 [==============================] - 1s - loss: 0.7945 - val_loss: 0.7183
Epoch 2/50
1124/1124 [==============================] - 1s - loss: 0.7930 - val_loss: 0.7167
Epoch 3/50
1124/1124 [==============================] - 1s - loss: 0.7918 - val_loss: 0.7151
Epoch 4/50
1124/1124 [==============================] - 1s - loss: 0.7897 - val_loss: 0.7135
Epoch 5/50
1124/1124 [==============================] - 1s - loss: 0.7876 - val_loss: 0.7120
Epoch 6/50
1124/1124 [==============================] - 1s - loss: 0.7866 - val_loss: 0.7104
Epoch 7/50
1124/1124 [==============================] - 1s - loss: 0.7850 - val_loss: 0.7088
Epoch 8/50
1124/1124 [==============================] - 1s - loss: 0.7836 - val_loss: 0.7072
Epoch 9/50
1124/1124 [==============================] - 1s - loss: 0.7822 - val_loss: 0.7057
Epoch 10/50
1124/1124 [==============================] - 1s - loss: 0.7801 - val_loss: 0.7041
Epoch 11/50
1124/1124

In [7]:
#diccionario para mantener los dominios adaptados
adapted = {}

for domain in domains:
    print "Adaptando %s" % domain
    X_tr = np.asarray(labeled[domain]['X_tr'][:, :dims].todense())
    X_ts = np.asarray(labeled[domain]['X_ts'][:, :dims].todense())
    
    tr_reps = encoder.predict(X_tr[:, :dims])
    ts_reps = encoder.predict(X_ts[:, :dims])
    
    adapted[domain] = {
        'X_tr': tr_reps,
        'X_ts': ts_reps
    }

Adaptando rio2016
Adaptando thevoice


In [8]:
df = pd.DataFrame(columns=dataframe_columns)

i = 0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapted[tgt]['X_ts']
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            svc_a = SVC(kernel='linear')
            svc_a = svc_a.fit(X_tr_a, y_tr)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['SDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            i+=1

print "Pruebas completadas."

Tarea 1 de 2
Cargando modelo existente.
Tarea 2 de 2
Cargando modelo existente.
Pruebas completadas.


In [9]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,SDA,r->t,rio2016,thevoice,16.081594,28.461538,12.379945
1,SDA,t->r,thevoice,rio2016,20.687361,25.000000,4.312639


In [10]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."


Guardando en scores/msda/twitter.csv
Resultados guardados.
